# function call

## 简介

function_call，顾名思义，通过给大模型提供 function 的说明描述，以及对应的入参出参 schema，让大模型输出 function 调用策略，结合多轮对话，以最终实现一个复杂的任务。
以下将以获取数据库中某类文件的数量为例子，通过调用千帆 Python SDK提供的 ERNIE-Bot 大模型以得到数据库中该语言的文件数量。

## 准备

本文使用了千帆 Python SDK中的 chat_completion 模块，该模块提供了与千帆对话引擎的交互接口，目前支持function call的模型有ERNIE-Bot与ERNIE-Bot4.0。

首先安装千帆 Python SDK

In [1]:
!pip install qianfan

如果你已经安装了千帆 Python SDK，我们建议进行一次升级

In [2]:
!pip install -U qianfan

初始化我们所需要的凭证

In [3]:
# 初始化LLM
import os

# qianfan sdk 鉴权
os.environ["QIANFAN_AK"] = "your_ak"
os.environ["QIANFAN_SK"] = "your_sk"

自定义一个给chat调用的函数，此处以获取数据库中特定语言撰写的文件数量为例

In [4]:
def get_file_num(language: str) -> str:
    """获取数据库中指定语言的代码文件数量"""
    language_low = language.lower()
    language_map = {
        "c/c++": 35,
        "java": 10,
        "javascript": 25,
        "python": 35,
        "go": 32,
    }
    return str(language_map.get(language_low, 0))

描述函数调用策略
其中，name为函数名称，description为函数描述，properties为入参schema，required为必填参数
properties支持json schema格式，具体参考[json schema](https://json-schema.org/learn/getting-started-step-by-step.html


In [5]:
func_list = [{
    "name": "get_file_num",  # 函数名称
    "description": "获取内部数据库中以某一编程语言编写的文件数量",  # 函数描述
    "parameters":{
        "type":"object",
        "properties":{  # 参数schema，如果参数为空，设为空字典即可
            "language":{  # 参数名称
                "type":"string",  # 参数类型
                "description": "代码所运用的编程语言，例如：python、c/c++、go、java"  # 参数描述
            }
        },
        "required":["language"]  # 必填参数（无默认值）
    }
}]

创建 chat_completion 对象,调用 do 方法进行交互

In [6]:
import qianfan,json

chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot")  # 指定模型，目前ERNIE-Bot和ERNIE-Bot4.0支持function call
query = "请帮我查询一下数据库中用go撰写的代码文件数量"
msgs = [{
    "role": "user",
    "content": query
}]
resp = chat_comp.do(
    messages=msgs,
    functions = func_list
)
print(resp['body']['result'])

可以发现，此时chat的反馈为null，我们尝试打印返回值

In [7]:
print(resp)

QfResponse(code=200, headers={'Access-Control-Allow-Headers': 'Content-Type', 'Access-Control-Allow-Origin': '*', 'Appid': '26217442', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json; charset=utf-8', 'Date': 'Thu, 11 Jan 2024 04:31:58 GMT', 'P3p': 'CP=" OTI DSP COR IVA OUR IND COM "', 'Server': 'Apache', 'Set-Cookie': 'BAIDUID=902FF76DAF722394B7DD3A38D36B64B8:FG=1; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2145916555; path=/; domain=.baidu.com; version=1', 'Statement': 'AI-generated', 'Vary': 'Accept-Encoding', 'X-Aipe-Self-Def': 'eb_total_tokens:199,prompt_tokens:164,id:as-uujczinwy6', 'X-Baidu-Request-Id': 'sdk-py-0.2.6-42cHWIuT1CVSs6Lz', 'X-Openapi-Server-Timestamp': '1704947516', 'Content-Length': '361'}, body={'id': 'as-uujczinwy6', 'object': 'chat.completion', 'created': 1704947518, 'result': '', 'is_truncated': False, 'need_clear_history': False, 'function_call': {'name': 'get_file_num', 'thoughts': '用户想要查询数据库中用go撰写的代码文件数量，我可以使用get_fi

从thoughts可见chat确定需要调用函数get_file_num，我们需要将反馈作为function输入来进行二次对话

In [8]:
if func_call_result := resp.get("function_call"):
    # 获取函数名称、入参及返回值
    func_name = func_call_result["name"]
    language = json.loads(func_call_result["arguments"]).get("language")
    func_resp = get_file_num(language)
    
    # 将函数的入参与返回值组合成json字符串
    func_content = json.dumps({
        "language":language,
        "description":func_resp
    })
    
    # 创建新的消息
    msgs.extend([{
        "role":"assistant",
        # "content":None,  # function_call非空时，content为空
        "function_call": func_call_result  # 将函数返回值作为输入
    },{
        "role":"function",
        "name":func_name,
        "content":func_content
    }])
    
    # 再次调用chat_completion
    second_resp = chat_comp.do(
        messages=msgs,
        functions=func_list
    )
    
    print(second_resp['body']['result'])

根据查询，数据库中用Go撰写的代码文件数量为32个。如果您还有其他问题或需要进一步了解各文件的情况，请随时告诉我。
